In [1]:
import os
import librosa
import pandas as pd
import ast
import soundfile as sf

In [2]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [3]:
def get_audio_path(audio_dir, track_id):
    """
    Return the path to the mp3 given the directory where the audio is stored
    and the track ID.

    Examples
    --------
    >>> import utils
    >>> AUDIO_DIR = os.environ.get('AUDIO_DIR')
    >>> utils.get_audio_path(AUDIO_DIR, 2)
    '../data/fma_small/000/000002.mp3'

    """
    tid_str = '{:06d}'.format(track_id)
    return os.path.join(audio_dir, tid_str[:3], tid_str + '.mp3')

In [4]:
AUDIO_DIR = 'fma_medium/'
tracks = load('tracks.csv')

os.mkdir('Samples')

medium = tracks['set', 'subset'] <= 'medium'

y_medium = tracks.loc[medium, ('track', 'genre_top')]

sr = 44100

In [10]:
#Saves only requested genres
error_list = []
genres_required = ["Hip-Hop", "Pop", "Rock", "Folk", "Jazz", "Blues", "Classical", "Country", "Reggae"]
for track_id, genre in y_medium.iteritems():
    if genre in genres_required:
        if not os.path.exists('Samples/'+genre):
            try:
                os.mkdir('Samples/'+genre)
            except FileNotFoundError:
                print("===============================================================================================================================")

        mp3_filename = get_audio_path(AUDIO_DIR, track_id)
        out_wav_filename = 'Samples/'+genre+'/'+str(track_id)+'.wav'
        in_wav_filename = out_wav_filename
        cmd = 'ffmpeg -hide_banner -loglevel panic -i ' + mp3_filename + ' ' + in_wav_filename
        print("excuting conversion: "+cmd)
        os.system(cmd)

        ## We could just have ffmpeg do the full conversion, but we'll let librosa
        ## apply its own defaults by reading & writing
        print("reading ",in_wav_filename)
    
        try:        
            data, sr = librosa.load(in_wav_filename, sr=sr, mono=True)

            print("writing ",out_wav_filename)
            sf.write(out_wav_filename,data,sr)
        except Exception as e:
            print("===============================================================================================================================")
            #error_list.append[e]
        

/tmp/ipykernel_15437/2349208370.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for track_id, genre in y_medium.iteritems():


excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000002.mp3 Samples/Hip-Hop/2.wav
reading  Samples/Hip-Hop/2.wav
writing  Samples/Hip-Hop/2.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000003.mp3 Samples/Hip-Hop/3.wav
reading  Samples/Hip-Hop/3.wav
writing  Samples/Hip-Hop/3.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000005.mp3 Samples/Hip-Hop/5.wav
reading  Samples/Hip-Hop/5.wav
writing  Samples/Hip-Hop/5.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000010.mp3 Samples/Pop/10.wav
reading  Samples/Pop/10.wav
writing  Samples/Pop/10.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000134.mp3 Samples/Hip-Hop/134.wav
reading  Samples/Hip-Hop/134.wav
writing  Samples/Hip-Hop/134.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000136.mp3 Samples/Rock/136.wav
reading  Samples/Rock/136.wav
writing  Samples

reading  Samples/Rock/574.wav
writing  Samples/Rock/574.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000583.mp3 Samples/Hip-Hop/583.wav
reading  Samples/Hip-Hop/583.wav
writing  Samples/Hip-Hop/583.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000584.mp3 Samples/Hip-Hop/584.wav
reading  Samples/Hip-Hop/584.wav
writing  Samples/Hip-Hop/584.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000585.mp3 Samples/Hip-Hop/585.wav
reading  Samples/Hip-Hop/585.wav
writing  Samples/Hip-Hop/585.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000590.mp3 Samples/Jazz/590.wav
reading  Samples/Jazz/590.wav
writing  Samples/Jazz/590.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000591.mp3 Samples/Jazz/591.wav
reading  Samples/Jazz/591.wav
writing  Samples/Jazz/591.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000

reading  Samples/Pop/823.wav
writing  Samples/Pop/823.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000824.mp3 Samples/Rock/824.wav
reading  Samples/Rock/824.wav
writing  Samples/Rock/824.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000825.mp3 Samples/Rock/825.wav
reading  Samples/Rock/825.wav
writing  Samples/Rock/825.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000826.mp3 Samples/Rock/826.wav
reading  Samples/Rock/826.wav
writing  Samples/Rock/826.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000829.mp3 Samples/Rock/829.wav
reading  Samples/Rock/829.wav
writing  Samples/Rock/829.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000830.mp3 Samples/Rock/830.wav
reading  Samples/Rock/830.wav
writing  Samples/Rock/830.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/000/000831.mp3 Samples/Jazz/831.wav


reading  Samples/Jazz/1064.wav
writing  Samples/Jazz/1064.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/001/001087.mp3 Samples/Rock/1087.wav
reading  Samples/Rock/1087.wav
writing  Samples/Rock/1087.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/001/001102.mp3 Samples/Rock/1102.wav
reading  Samples/Rock/1102.wav
writing  Samples/Rock/1102.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/001/001105.mp3 Samples/Rock/1105.wav
reading  Samples/Rock/1105.wav
writing  Samples/Rock/1105.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/001/001106.mp3 Samples/Rock/1106.wav
reading  Samples/Rock/1106.wav
writing  Samples/Rock/1106.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/001/001107.mp3 Samples/Rock/1107.wav
reading  Samples/Rock/1107.wav
writing  Samples/Rock/1107.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/001/001108.mp3 Sa

...

reading  Samples/Rock/149417.wav
writing  Samples/Rock/149417.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149452.mp3 Samples/Rock/149452.wav
reading  Samples/Rock/149452.wav
writing  Samples/Rock/149452.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149453.mp3 Samples/Rock/149453.wav
reading  Samples/Rock/149453.wav
writing  Samples/Rock/149453.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149488.mp3 Samples/Rock/149488.wav
reading  Samples/Rock/149488.wav
writing  Samples/Rock/149488.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149489.mp3 Samples/Rock/149489.wav
reading  Samples/Rock/149489.wav
writing  Samples/Rock/149489.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149514.mp3 Samples/Rock/149514.wav
reading  Samples/Rock/149514.wav
writing  Samples/Rock/149514.wav
excuting conversion: ffmpeg -hide_banner -loglevel pan

excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149703.mp3 Samples/Classical/149703.wav
reading  Samples/Classical/149703.wav
writing  Samples/Classical/149703.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149705.mp3 Samples/Classical/149705.wav
reading  Samples/Classical/149705.wav
writing  Samples/Classical/149705.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149706.mp3 Samples/Classical/149706.wav
reading  Samples/Classical/149706.wav
writing  Samples/Classical/149706.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149707.mp3 Samples/Classical/149707.wav
reading  Samples/Classical/149707.wav
writing  Samples/Classical/149707.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/149/149708.mp3 Samples/Classical/149708.wav
reading  Samples/Classical/149708.wav
writing  Samples/Classical/149708.wav
excuting conversion: ffmpeg -hide_banner -log

reading  Samples/Rock/150481.wav
writing  Samples/Rock/150481.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/150/150482.mp3 Samples/Rock/150482.wav
reading  Samples/Rock/150482.wav
writing  Samples/Rock/150482.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/150/150568.mp3 Samples/Rock/150568.wav
reading  Samples/Rock/150568.wav
writing  Samples/Rock/150568.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/150/150572.mp3 Samples/Rock/150572.wav
reading  Samples/Rock/150572.wav
writing  Samples/Rock/150572.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/150/150573.mp3 Samples/Rock/150573.wav
reading  Samples/Rock/150573.wav
writing  Samples/Rock/150573.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/150/150574.mp3 Samples/Rock/150574.wav
reading  Samples/Rock/150574.wav
writing  Samples/Rock/150574.wav
excuting conversion: ffmpeg -hide_banner -loglevel pan

reading  Samples/Hip-Hop/152544.wav
writing  Samples/Hip-Hop/152544.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/152/152545.mp3 Samples/Hip-Hop/152545.wav
reading  Samples/Hip-Hop/152545.wav
writing  Samples/Hip-Hop/152545.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/152/152624.mp3 Samples/Rock/152624.wav
reading  Samples/Rock/152624.wav
writing  Samples/Rock/152624.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/152/152625.mp3 Samples/Rock/152625.wav
reading  Samples/Rock/152625.wav
writing  Samples/Rock/152625.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/152/152628.mp3 Samples/Rock/152628.wav
reading  Samples/Rock/152628.wav
writing  Samples/Rock/152628.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/152/152632.mp3 Samples/Rock/152632.wav
reading  Samples/Rock/152632.wav
writing  Samples/Rock/152632.wav
excuting conversion: ffmpeg -hide_banne

reading  Samples/Hip-Hop/154299.wav
writing  Samples/Hip-Hop/154299.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/154/154300.mp3 Samples/Hip-Hop/154300.wav
reading  Samples/Hip-Hop/154300.wav
writing  Samples/Hip-Hop/154300.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/154/154301.mp3 Samples/Hip-Hop/154301.wav
reading  Samples/Hip-Hop/154301.wav
writing  Samples/Hip-Hop/154301.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/154/154302.mp3 Samples/Hip-Hop/154302.wav
reading  Samples/Hip-Hop/154302.wav
writing  Samples/Hip-Hop/154302.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/154/154303.mp3 Samples/Hip-Hop/154303.wav
reading  Samples/Hip-Hop/154303.wav
writing  Samples/Hip-Hop/154303.wav
excuting conversion: ffmpeg -hide_banner -loglevel panic -i fma_medium/154/154304.mp3 Samples/Hip-Hop/154304.wav
reading  Samples/Hip-Hop/154304.wav
writing  Samples/Hip-Hop/154304.wav
exc

In [ ]:
print(error_list)